In [70]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import warnings


warnings.filterwarnings("ignore")

## Dataset

In [21]:

dataset = pd.read_json("sarcasm.json")

## Cleaning

In [22]:
dataset.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [23]:
dataset.tail()

,is_sarcastic,headline,article_link
28614,1,jews to celebrate rosh hashasha or something,https://www.theonion.com/jews-to-celebrate-ros...
28615,1,internal affairs investigator disappointed con...,https://local.theonion.com/internal-affairs-in...
28616,0,the most beautiful acceptance speech this week...,https://www.huffingtonpost.com/entry/andrew-ah...
28617,1,mars probe destroyed by orbiting spielberg-gat...,https://www.theonion.com/mars-probe-destroyed-...
28618,1,dad clarifies this not a food stop,https://www.theonion.com/dad-clarifies-this-no...


Drop article_link column

In [24]:
df = dataset.drop(["article_link"], axis = 1)

## Tokenizer

In [39]:
VOCAB_SIZE = 30000
tokenizer = Tokenizer(num_words = VOCAB_SIZE, oov_token = "OOV")
tokenizer.fit_on_texts(df.headline.values)
headline_sequences = tokenizer.texts_to_sequences(df.headline.values)

Check dictionary

In [62]:
tokenizer.word_index

{'OOV': 1,
 'to': 2,
 'of': 3,
 'the': 4,
 'in': 5,
 'for': 6,
 'a': 7,
 'on': 8,
 'and': 9,
 'with': 10,
 'is': 11,
 'new': 12,
 'trump': 13,
 'man': 14,
 'at': 15,
 'from': 16,
 'about': 17,
 'by': 18,
 'after': 19,
 'you': 20,
 'this': 21,
 'out': 22,
 'up': 23,
 'be': 24,
 'as': 25,
 'that': 26,
 'it': 27,
 'how': 28,
 'not': 29,
 'he': 30,
 'his': 31,
 'are': 32,
 'your': 33,
 'just': 34,
 'what': 35,
 'all': 36,
 'who': 37,
 'has': 38,
 'will': 39,
 'report': 40,
 'into': 41,
 'more': 42,
 'one': 43,
 'have': 44,
 'year': 45,
 'over': 46,
 'why': 47,
 'day': 48,
 'u': 49,
 'area': 50,
 'woman': 51,
 'can': 52,
 's': 53,
 'says': 54,
 'donald': 55,
 'time': 56,
 'first': 57,
 'like': 58,
 'no': 59,
 'her': 60,
 'get': 61,
 'off': 62,
 'old': 63,
 "trump's": 64,
 'life': 65,
 'now': 66,
 'people': 67,
 "'": 68,
 'an': 69,
 'house': 70,
 'still': 71,
 'obama': 72,
 'white': 73,
 'back': 74,
 'make': 75,
 'was': 76,
 'than': 77,
 'women': 78,
 'if': 79,
 'down': 80,
 'when': 81,
 'i'

Find maximum length of sublists

In [63]:
headline_sequences[0]

[16004, 355, 3167, 7474, 2644, 3, 661, 1119]

In [64]:
len_each_sublist = [len(sublist) for sublist in headline_sequences]

In [66]:
max(len_each_sublist)

152

Maximum length of sublists is 152

In [67]:
MAX_LEN = 152

### Padding

In [72]:
padded_headline_seq = pad_sequences(headline_sequences, truncating = 'post', padding = 'post', maxlen = MAX_LEN)

## Train Test Split

### Set random state 

In [74]:
RS  = 13

In [75]:
X_train, X_test, y_train, y_test = train_test_split(padded_headline_seq, df.is_sarcastic, test_size = 0.2, random_state = RS)

## Model

In [79]:
X_train.shape

(22895, 152)

In [77]:
EMBEDDING_DIM = 150

In [80]:
def build_model():
    model = keras.Sequential()
    model.add(keras.layers.Input((X_train.shape[1],)))
    model.add(keras.layers.Dense(100, activation = 'relu'))
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model


In [81]:
EPOCHS = 100

In [82]:
model = build_model()
model.fit(X_train,y_train, epochs = EPOCHS)

Epoch 1/100
716/716 [==============================] - 4s 2ms/step - loss: 77.8845 - accuracy: 0.5204
Epoch 2/100
716/716 [==============================] - 2s 3ms/step - loss: 44.4617 - accuracy: 0.5288
Epoch 3/100
716/716 [==============================] - 2s 2ms/step - loss: 34.0318 - accuracy: 0.5323
Epoch 4/100
716/716 [==============================] - 2s 2ms/step - loss: 27.3815 - accuracy: 0.5366
Epoch 5/100
716/716 [==============================] - 2s 2ms/step - loss: 23.4641 - accuracy: 0.5426
Epoch 6/100
716/716 [==============================] - 2s 3ms/step - loss: 19.8866 - accuracy: 0.5452
Epoch 7/100
716/716 [==============================] - 3s 4ms/step - loss: 16.0032 - accuracy: 0.5421
Epoch 8/100
716/716 [==============================] - 2s 3ms/step - loss: 13.8529 - accuracy: 0.5476
Epoch 9/100
716/716 [==============================] - 2s 3ms/step - loss: 11.2659 - accuracy: 0.5473
Epoch 10/100
716/716 [==============================] - 2s 3ms/step - loss: 9.3391